#### Simple Gen AI APP Using Langchain

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

# Groq + Langsmith Keys
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [4]:
pip install langchain_community

     ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
     - -------------------------------------- 0.1/2.5 MB 1.8 MB/s eta 0:00:02
     ----------- ---------------------------- 0.7/2.5 MB 7.7 MB/s eta 0:00:01
     ---------------------------------------  2.5/2.5 MB 17.6 MB/s eta 0:00:01
     ---------------------------------------- 2.5/2.5 MB 14.7 MB/s eta 0:00:00
  Using cached aiohttp-3.12.15-cp310-cp310-win_amd64.whl (452 kB)
  Using cached langchain-0.3.27-py3-none-any.whl (1.0 MB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl (45 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl (8.1 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
  Using cached sqlalchemy-2.0.43-cp310-cp310-win_amd64.whl (2.1 MB)
  Using cached yarl-1.20.1-cp310-cp310-win_amd64.whl (86 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
  Using cached multidict-6.6.4-cp310-cp310-win_amd64.whl (46 kB)
  Using cached propcache-0.3.2-cp310-cp310-win_am


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
pip install bs4

  Using cached beautifulsoup4-4.13.5-py3-none-any.whl (105 kB)
  Using cached soupsieve-2.8-py3-none-any.whl (36 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:

## Data Ingestion
from langchain_community.document_loaders import WebBaseLoader
#loader = WebBaseLoader("https://docs.smith.langchain.com/tutorials/Administrators/manage_spend")
#loader = WebBaseLoader("https://python.langchain.com/docs/langsmith/guides/manage_spend/")
loader = WebBaseLoader("https://www.geeksforgeeks.org/artificial-intelligence/the-complete-langchain-ecosystem/")


docs = loader.load()

In [31]:
## Split documents
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

In [32]:
pip install sentence_transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
pip install faiss-cpu


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
      --------------------------------------- 0.3/18.2 MB 8.9 MB/s eta 0:00:03
     --- ------------------------------------ 1.4/18.2 MB 15.1 MB/s eta 0:00:02
     ------ --------------------------------- 2.8/18.2 MB 19.9 MB/s eta 0:00:01
     ------- -------------------------------- 3.4/18.2 MB 17.9 MB/s eta 0:00:01
     -------- ------------------------------- 3.9/18.2 MB 17.8 MB/s eta 0:00:01
     --------- ------------------------------ 4.2/18.2 MB 15.7 MB/s eta 0:00:01
     --------- ------------------------------ 4.2/18.2 MB 15.7 MB/s eta 0:00:01
     --------- ------------------------------ 4.3/18.2 MB 11.5 MB/s eta 0:00:02
     ---------- ----------------------------- 5.0/18.2 MB 11.8 MB/s eta 0:00:02
     -------------- ------------------------- 6.8/18.2 MB 14.5 MB/s eta 0:00:01
     ---------------- ----------------------- 7.4/18.2 MB 14.3 MB/s eta 0:00:01
     ----------------- ---------------------- 8.

In [33]:
## Embeddings (HuggingFace instead of OpenAI)
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstoredb = FAISS.from_documents(documents, embeddings)

In [34]:
## Query Test
query = "What is LangChain Core and what roles does it serve?"
result = vectorstoredb.similarity_search(query)
print("Retrieved Context:\n", result[0].page_content)

Retrieved Context:
 LangChain is a framework that simplifies the process of building applications using Large Language Models (LLMs). It provides a set of modular building blocks that developers can use to design, integrate and deploy intelligent systems. The architecture of LangChain is structured in a way that separates the core logic, integration layers and advanced deployment tools. This layered approach makes LangChain both flexible for experimentation and robust for production use.LangChain ArchitectureCore Architecture of LangChainLet’s see how LangChain is structured from its core modules to advanced tools:1. LangChain Core (langchain-core)Provides the fundamental abstractions of the framework.Includes prompts, messages, models, chains, memory and output parsers.Does not depend on external providers, ensuring a lightweight base.Role: The foundation on which other layers are built.2. Main LangChain Package (langchain)Builds on langchain-core by offering higher-level workflows.Pr

In [35]:

## LLM (Groq)
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.7)

In [36]:
## Retrieval Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>
"""
)

document_chain = create_stuff_documents_chain(llm, prompt)

In [37]:
from langchain.chains import create_retrieval_chain
retriever = vectorstoredb.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [40]:
## Final Response
response = retrieval_chain.invoke({"input": "What key components of LangChain?"})
print("\nAnswer:\n", response["answer"])


Answer:
 Based on the provided context, here are the 7 key components of the LangChain framework:

1. **LangChain Core (langchain-core)**: Provides the fundamental abstractions of the framework, including prompts, messages, models, chains, memory, and output parsers.
2. **Main LangChain Package (langchain)**: Offers higher-level workflows, orchestration for chains and agents, and remains model-agnostic.
3. **Integration Packages**: Independent connectors for external tools and services, such as OpenAI models and Pinecone vector database.
4. **LangChain Community (langchain-community)**: A collection of community-maintained integrations that provides connectors for services not officially covered.
5. **LangGraph**: A framework for building stateful applications as computational graphs, supporting memory management, streaming outputs, and complex decision flows.
6. **LangServe**: Used for deploying LangChain applications as APIs, offering a straightforward way to expose chains or agents